<a href="https://colab.research.google.com/github/noamsiegel/Certifications/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [32]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-02-26 20:36:28--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.5’

book-crossings.zip. 100%[===================>]  24.88M  99.8MB/s    in 0.2s    

2023-02-26 20:36:29 (99.8 MB/s) - ‘book-crossings.zip.5’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Books.csv            y

replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: BX-Users.csv            


In [33]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [38]:
#@title Cleaning the data

# drop null authors
df_books_cleaned = df_books.loc[~df_books.author.isnull()]
df_books_cleaned.isnull().sum()

# drop users without over 200 ratings
active_users = df_ratings.user.value_counts()[df_ratings.user.value_counts()>=200].index

# drop books that have fewer than 100 ratings
popular_books = df_ratings.isbn.value_counts()[df_ratings.isbn.value_counts()>=100].index

# make official data frames for books and ratings
df_ratings_cleaned = df_ratings.loc[(df_ratings.user.isin(active_users))]
df_books_cleaned = df_books_cleaned.loc[(df_books_cleaned.isbn.isin(popular_books))]

# merge dataframes
df_merged = pd.merge(left=df_ratings_cleaned,right= df_books_cleaned, on='isbn', how='inner')

# drop duplicates in merged dataframe
df_unique = df_merged.drop_duplicates(["title","user"])

# create pivot and sort it
pivot = df_unique.pivot(index='title', columns='user', values='rating').fillna(0)
pivot = pivot.sort_index()

# create model
input = pivot.values
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(input)

NearestNeighbors(algorithm='brute', metric='cosine')

In [40]:
#@title Recommended Books Function

def get_recommends(book = "Where the Heart Is (Oprah's Book Club (Paperback))"):
  
  # creating five recommened books
  dist, suggested_books = model.kneighbors(pivot.loc[book].values.reshape(1,-1), n_neighbors=6)
  
  books = pivot.iloc[suggested_books[0]].index.values
  result = list(zip(books, dist[0]))
  result[0] = result[0][0]
  result[1] = sorted([[book,distance] for book,distance in result[1:]], key=lambda x: -x[1])[:4]
  recommended_books = result[:2]
  
  return recommended_books

In [41]:
#@title Challenge Testing
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

# function testing
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075]]]
You passed the challenge! 🎉🎉🎉🎉🎉
